In [91]:
import pandas as pd
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
from qdrant_client.models import Distance, VectorParams, PointStruct,Batch,Payload
import uuid

In [92]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
client = QdrantClient(url="http://localhost:6333")
collection_name = "no_coordinates_collection"

/home/davis/miniconda3/envs/AI/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [93]:
# Replace 'path_to_yelp_academic_dataset_business.json' with the actual path to your JSON file
features = ['business_id', 'name', 'address', 'city','postal_code', 'categories', 'attributes', 'longitude', 'latitude', 'stars', 'review_count', 'hours']
df = pd.read_json('archive/yelp_academic_dataset_business.json', lines=True)
df_cali= df[df['state'] == 'CA']
df_features = df_cali[features]
df_features

,business_id,name,address,city,postal_code,categories,attributes,longitude,latitude,stars,review_count,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,93101,"Doctors, Traditional Chinese Medicine, Naturop...",{'ByAppointmentOnly': 'True'},-119.711197,34.426679,5.0,7,None
26,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,93101,"Women's Clothing, Accessories, Children's Clot...","{'BusinessAcceptsCreditCards': 'True', 'BikePa...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'..."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,93101,"Food, Restaurants, Salad, Coffee & Tea, Breakf...","{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,825 Cacique St,Santa Barbara,93103,"Towing, Automotive, Body Shops","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '..."
120,bYjnX_J1bHZob10DoSFkqQ,Tinkle Belle Diaper Service,,Santa Barbara,93101,"Parenting Classes, Maternity Wear, Fashion, Ed...","{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",-119.710749,34.420334,5.0,17,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,93105,"Event Planning & Services, Wine Tasting Room, ...","{'BusinessParking': '{'garage': False, 'street...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,93117,"Restaurants, Burgers, Food, Ice Cream & Frozen...","{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",-119.856886,34.412966,2.5,12,None
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","1805 State St, Ste B",Santa Barbara,93101,"Shopping, Eyewear & Opticians, Health & Medica...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '..."
150290,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,93105,"Restaurants, Pizza, Food","{'NoiseLevel': 'u'average'', 'RestaurantsGoodF...",-119.739681,34.440689,4.0,299,"{'Monday': '15:0-21:0', 'Tuesday': '15:0-21:0'..."


In [94]:
# Concatenating address, city, and postal code with a comma and space
df_features['full_address'] = df_features.apply(lambda x: f"{x['address']} {x['city']} {x['postal_code']}", axis=1)
df_features


/tmp/ipykernel_3060536/2443034162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['full_address'] = df_features.apply(lambda x: f"{x['address']} {x['city']} {x['postal_code']}", axis=1)


,business_id,name,address,city,postal_code,categories,attributes,longitude,latitude,stars,review_count,hours,full_address
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,93101,"Doctors, Traditional Chinese Medicine, Naturop...",{'ByAppointmentOnly': 'True'},-119.711197,34.426679,5.0,7,None,"1616 Chapala St, Ste 2 Santa Barbara 93101"
26,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,93101,"Women's Clothing, Accessories, Children's Clot...","{'BusinessAcceptsCreditCards': 'True', 'BikePa...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St Santa Barbara 93101
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,93101,"Food, Restaurants, Salad, Coffee & Tea, Breakf...","{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C Santa Barbara 93101"
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,825 Cacique St,Santa Barbara,93103,"Towing, Automotive, Body Shops","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St Santa Barbara 93103
120,bYjnX_J1bHZob10DoSFkqQ,Tinkle Belle Diaper Service,,Santa Barbara,93101,"Parenting Classes, Maternity Wear, Fashion, Ed...","{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",-119.710749,34.420334,5.0,17,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",Santa Barbara 93101
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,93105,"Event Planning & Services, Wine Tasting Room, ...","{'BusinessParking': '{'garage': False, 'street...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St Santa Barbara 93105
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,93117,"Restaurants, Burgers, Food, Ice Cream & Frozen...","{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",-119.856886,34.412966,2.5,12,None,900 Embarcadero Del Mar Isla Vista 93117
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","1805 State St, Ste B",Santa Barbara,93101,"Shopping, Eyewear & Opticians, Health & Medica...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B Santa Barbara 93101"
150290,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,93105,"Restaurants, Pizza, Food","{'NoiseLevel': 'u'average'', 'RestaurantsGoodF...",-119.739681,34.440689,4.0,299,"{'Monday': '15:0-21:0', 'Tuesday': '15:0-21:0'...",3534 State St Santa Barbara 93105


In [95]:

# Prepare an empty list to store combined embeddings
combined_embeddings = []

# Loop through each row in the DataFrame
for index, row in tqdm(df_features.iterrows(), total=df_features.shape[0], desc="Encoding rows"):
    # Generate embeddings for each row
    address_embedding = model.encode(row['full_address'])
    coordinates = np.array([row['longitude'], row['latitude']])
    # combined_embedding = np.concatenate((address_embedding, coordinates))
    combined_embeddings.append(address_embedding)

df_features['combined_embeddings'] = combined_embeddings
df_features

Encoding rows: 100%|██████████| 5203/5203 [00:45<00:00, 114.11it/s]


,business_id,name,address,city,postal_code,categories,attributes,longitude,latitude,stars,review_count,hours,full_address,combined_embeddings
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,93101,"Doctors, Traditional Chinese Medicine, Naturop...",{'ByAppointmentOnly': 'True'},-119.711197,34.426679,5.0,7,None,"1616 Chapala St, Ste 2 Santa Barbara 93101","[-0.022869986, -0.02613301, -0.10245921, -0.03..."
26,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,93101,"Women's Clothing, Accessories, Children's Clot...","{'BusinessAcceptsCreditCards': 'True', 'BikePa...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St Santa Barbara 93101,"[-0.023923889, -0.060360875, -0.094587356, -0...."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,93101,"Food, Restaurants, Salad, Coffee & Tea, Breakf...","{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C Santa Barbara 93101","[0.0521911, -0.029908517, -0.07463628, -0.0408..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,825 Cacique St,Santa Barbara,93103,"Towing, Automotive, Body Shops","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St Santa Barbara 93103,"[-0.0021292954, -0.02788783, -0.07666405, -0.0..."
120,bYjnX_J1bHZob10DoSFkqQ,Tinkle Belle Diaper Service,,Santa Barbara,93101,"Parenting Classes, Maternity Wear, Fashion, Ed...","{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",-119.710749,34.420334,5.0,17,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",Santa Barbara 93101,"[-0.0054948414, -0.058293823, -0.100457184, -0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,93105,"Event Planning & Services, Wine Tasting Room, ...","{'BusinessParking': '{'garage': False, 'street...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St Santa Barbara 93105,"[0.0065628192, 0.016845139, -0.071850985, -0.0..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,93117,"Restaurants, Burgers, Food, Ice Cream & Frozen...","{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",-119.856886,34.412966,2.5,12,None,900 Embarcadero Del Mar Isla Vista 93117,"[-0.012040307, 0.048302714, -0.028739816, -0.0..."
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","1805 State St, Ste B",Santa Barbara,93101,"Shopping, Eyewear & Opticians, Health & Medica...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B Santa Barbara 93101","[0.016532987, -0.015368396, -0.092512, -0.0193..."
150290,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,93105,"Restaurants, Pizza, Food","{'NoiseLevel': 'u'average'', 'RestaurantsGoodF...",-119.739681,34.440689,4.0,299,"{'Monday': '15:0-21:0', 'Tuesday': '15:0-21:0'...",3534 State St Santa Barbara 93105,"[0.04091528, -0.035807077, -0.109056726, -0.03..."


In [96]:
df_features = df_features.drop(columns=['address', 'longitude', 'latitude', 'city', 'postal_code','attributes' ])
df_features

,business_id,name,categories,stars,review_count,hours,full_address,combined_embeddings
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","Doctors, Traditional Chinese Medicine, Naturop...",5.0,7,None,"1616 Chapala St, Ste 2 Santa Barbara 93101","[-0.022869986, -0.02613301, -0.10245921, -0.03..."
26,noByYNtDLQAra9ccqxdfDw,H&M,"Women's Clothing, Accessories, Children's Clot...",3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St Santa Barbara 93101,"[-0.023923889, -0.060360875, -0.094587356, -0...."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"Food, Restaurants, Salad, Coffee & Tea, Breakf...",4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C Santa Barbara 93101","[0.0521911, -0.029908517, -0.07463628, -0.0408..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,"Towing, Automotive, Body Shops",4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St Santa Barbara 93103,"[-0.0021292954, -0.02788783, -0.07666405, -0.0..."
120,bYjnX_J1bHZob10DoSFkqQ,Tinkle Belle Diaper Service,"Parenting Classes, Maternity Wear, Fashion, Ed...",5.0,17,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",Santa Barbara 93101,"[-0.0054948414, -0.058293823, -0.100457184, -0..."
...,...,...,...,...,...,...,...,...
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,"Event Planning & Services, Wine Tasting Room, ...",5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St Santa Barbara 93105,"[0.0065628192, 0.016845139, -0.071850985, -0.0..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,"Restaurants, Burgers, Food, Ice Cream & Frozen...",2.5,12,None,900 Embarcadero Del Mar Isla Vista 93117,"[-0.012040307, 0.048302714, -0.028739816, -0.0..."
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","Shopping, Eyewear & Opticians, Health & Medica...",4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B Santa Barbara 93101","[0.016532987, -0.015368396, -0.092512, -0.0193..."
150290,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,"Restaurants, Pizza, Food",4.0,299,"{'Monday': '15:0-21:0', 'Tuesday': '15:0-21:0'...",3534 State St Santa Barbara 93105,"[0.04091528, -0.035807077, -0.109056726, -0.03..."


In [97]:
print('dimensions',len(df_features['combined_embeddings'][0]))

dimensions 384


In [98]:
df_features

,business_id,name,categories,stars,review_count,hours,full_address,combined_embeddings
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","Doctors, Traditional Chinese Medicine, Naturop...",5.0,7,None,"1616 Chapala St, Ste 2 Santa Barbara 93101","[-0.022869986, -0.02613301, -0.10245921, -0.03..."
26,noByYNtDLQAra9ccqxdfDw,H&M,"Women's Clothing, Accessories, Children's Clot...",3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St Santa Barbara 93101,"[-0.023923889, -0.060360875, -0.094587356, -0...."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"Food, Restaurants, Salad, Coffee & Tea, Breakf...",4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C Santa Barbara 93101","[0.0521911, -0.029908517, -0.07463628, -0.0408..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,"Towing, Automotive, Body Shops",4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St Santa Barbara 93103,"[-0.0021292954, -0.02788783, -0.07666405, -0.0..."
120,bYjnX_J1bHZob10DoSFkqQ,Tinkle Belle Diaper Service,"Parenting Classes, Maternity Wear, Fashion, Ed...",5.0,17,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",Santa Barbara 93101,"[-0.0054948414, -0.058293823, -0.100457184, -0..."
...,...,...,...,...,...,...,...,...
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,"Event Planning & Services, Wine Tasting Room, ...",5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St Santa Barbara 93105,"[0.0065628192, 0.016845139, -0.071850985, -0.0..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,"Restaurants, Burgers, Food, Ice Cream & Frozen...",2.5,12,None,900 Embarcadero Del Mar Isla Vista 93117,"[-0.012040307, 0.048302714, -0.028739816, -0.0..."
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","Shopping, Eyewear & Opticians, Health & Medica...",4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B Santa Barbara 93101","[0.016532987, -0.015368396, -0.092512, -0.0193..."
150290,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,"Restaurants, Pizza, Food",4.0,299,"{'Monday': '15:0-21:0', 'Tuesday': '15:0-21:0'...",3534 State St Santa Barbara 93105,"[0.04091528, -0.035807077, -0.109056726, -0.03..."


In [105]:
response = client.create_collection(
    collection_name=f"{collection_name}",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)
print(response)

True


In [106]:

def hash_to_uuid(unique_string):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, unique_string))

def process_catagoires(categories):
    return categories.split(', ') if categories else []
# Adjust the Payload construction if needed
def create_point(row):
    return PointStruct(
        id=hash_to_uuid(row['business_id']),
        vector=np.array(row['combined_embeddings']).tolist(),  # Ensure it's converted to list properly
        payload={
            'name' : row['name'],
            'categories' : process_catagoires(row['categories']) ,  # Assuming categories is a single string of categories
            'stars' : float(row['stars']),  # Ensure it's a float
            'review_count' : int(row['review_count']),  # Ensure it's an int
            'hours' : row['hours'],  # Ensure hours are in a JSON serializable format
            'full_address': row['full_address']
        }
        )

points = []
for index, row in tqdm(df_features.iterrows()):   
    point = create_point(row)
    points.append(point)


print(len(points))



5203it [00:00, 12633.56it/s]

5203


In [107]:
print(points[0])

id='92fc196f-0e3c-5f24-9ebb-09b821e99fce' vector=[-0.022869985550642014, -0.026133010163903236, -0.10245920717716217, -0.030436230823397636, -0.12694787979125977, 0.001102060661651194, -0.07925020158290863, 0.0183518435806036, 0.02975236251950264, -0.0031201201491057873, 0.05882612615823746, -0.037977185100317, -0.005459301173686981, -0.12082471698522568, 0.03329894691705704, -0.032697003334760666, 0.02977568656206131, -0.047616615891456604, 0.11756628751754761, 0.009724081493914127, 0.042324453592300415, 0.056712254881858826, -0.07340318709611893, -0.053371988236904144, 0.014153367839753628, 0.067670539021492, 0.008580266498029232, 0.021394500508904457, -0.040459755808115005, -0.06162893772125244, 0.08864090591669083, -0.028215980157256126, 0.05541468411684036, 0.015090013854205608, 0.06295850872993469, 0.007132223807275295, 0.01892266236245632, -0.009060355834662914, 0.08505146205425262, -0.000306760921375826, -0.051245518028736115, 0.014752336777746677, -0.008845747448503971, -0.011

In [108]:

def upload_batches(points, batch_size=100):

    total_batches = len(points) // batch_size + (1 if len(points) % batch_size > 0 else 0)

    for i in tqdm(range(total_batches), desc="Uploading batches"):
        start_index = i * batch_size
        end_index = start_index + batch_size
        batch_points = points[start_index:end_index]
 
        response = client.upsert(
            collection_name=f"{collection_name}",
            points=batch_points
        )
        print(f"Batch {i+1}/{total_batches} uploaded. Response: {response}")

    

In [109]:
upload_batches(points, batch_size=100)


Uploading batches:  13%|█▎        | 7/53 [00:00<00:00, 63.30it/s]

Batch 1/53 uploaded. Response: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 2/53 uploaded. Response: operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 3/53 uploaded. Response: operation_id=2 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 4/53 uploaded. Response: operation_id=3 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 5/53 uploaded. Response: operation_id=4 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 6/53 uploaded. Response: operation_id=5 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 7/53 uploaded. Response: operation_id=6 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 8/53 uploaded. Response: operation_id=7 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 9/53 uploaded. Response: operation_id=8 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 10/53 uploaded. Response: operation_id=9 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 11/53 uploaded. Response: operation_id=10 status=<UpdateStatus.COMPLETED: 'complete

Uploading batches:  26%|██▋       | 14/53 [00:00<00:00, 64.23it/s]

Batch 12/53 uploaded. Response: operation_id=11 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 13/53 uploaded. Response: operation_id=12 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 14/53 uploaded. Response: operation_id=13 status=<UpdateStatus.COMPLETED: 'completed'>


Uploading batches:  40%|███▉      | 21/53 [00:00<00:00, 65.57it/s]

...

Batch 27/53 uploaded. Response: operation_id=26 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 28/53 uploaded. Response: operation_id=27 status=<UpdateStatus.COMPLETED: 'completed'>


Uploading batches:  68%|██████▊   | 36/53 [00:00<00:00, 68.00it/s]

Batch 29/53 uploaded. Response: operation_id=28 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 30/53 uploaded. Response: operation_id=29 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 31/53 uploaded. Response: operation_id=30 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 32/53 uploaded. Response: operation_id=31 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 33/53 uploaded. Response: operation_id=32 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 34/53 uploaded. Response: operation_id=33 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 35/53 uploaded. Response: operation_id=34 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 36/53 uploaded. Response: operation_id=35 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 37/53 uploaded. Response: operation_id=36 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 38/53 uploaded. Response: operation_id=37 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 39/53 uploaded. Response: operation_id=38 status=<UpdateStatus.C

Uploading batches: 100%|██████████| 53/53 [00:00<00:00, 66.82it/s]

Batch 44/53 uploaded. Response: operation_id=43 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 45/53 uploaded. Response: operation_id=44 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 46/53 uploaded. Response: operation_id=45 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 47/53 uploaded. Response: operation_id=46 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 48/53 uploaded. Response: operation_id=47 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 49/53 uploaded. Response: operation_id=48 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 50/53 uploaded. Response: operation_id=49 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 51/53 uploaded. Response: operation_id=50 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 52/53 uploaded. Response: operation_id=51 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 53/53 uploaded. Response: operation_id=52 status=<UpdateStatus.COMPLETED: 'completed'>
